# High-level Keras (TF) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from common.params import *
from common.utils import *

Using TensorFlow backend.


In [2]:
# channels_last is faster
K.backend.set_image_data_format('channels_last')

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
# Channels should be last (otherwise slow)
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.1.1
Numpy:  1.13.3
Tensorflow:  1.4.0
tensorflow
channels_last
GPU:  ['Tesla K80']


In [4]:
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

In [5]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [6]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [7]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 829 ms, sys: 565 ms, total: 1.39 s
Wall time: 1.39 s


In [8]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 205 ms, sys: 0 ns, total: 205 ms
Wall time: 204 ms


In [9]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 146 ms, sys: 4.38 ms, total: 150 ms
Wall time: 150 ms


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 50)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 100)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 100)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 100)         0         
__________

In [11]:
%%time
# 241s
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 26s 514us/step - loss: 1.8262 - acc: 0.3318
Epoch 2/10
50000/50000 [==============================] - 24s 478us/step - loss: 1.3565 - acc: 0.5100
Epoch 3/10
50000/50000 [==============================] - 24s 478us/step - loss: 1.1200 - acc: 0.5996
Epoch 4/10
50000/50000 [==============================] - 24s 478us/step - loss: 0.9835 - acc: 0.6507
Epoch 5/10
50000/50000 [==============================] - 24s 479us/step - loss: 0.8590 - acc: 0.6977
Epoch 6/10
50000/50000 [==============================] - 24s 479us/step - loss: 0.7826 - acc: 0.7261
Epoch 7/10
50000/50000 [==============================] - ETA: 0s - loss: 0.7106 - acc: 0.750 - 24s 479us/step - loss: 0.7107 - acc: 0.7503
Epoch 8/10
50000/50000 [==============================] - 24s 479us/step - loss: 0.6460 - acc: 0.7709
Epoch 9/10
50000/50000 [==============================] - 24s 478us/step - loss: 0.5953 - acc: 0.7895
Epoch 10/10
50000/50000 [===================

In [12]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 529 ms, sys: 92.4 ms, total: 621 ms
Wall time: 1.39 s


In [13]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.773
